# Read data and functions

In [1106]:
import geopandas as gpd
import folium
import rioxarray as riox
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.linear_model import LassoCV
from numpy import arange
from geopandas import GeoDataFrame, GeoSeries

In [1107]:
pd.set_option('display.max_columns', None)

In [1108]:
neighborhoods_frankfurt = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\Halle\Hackathon local data\1 Land Prices\Land_Prices_Neighborhood_Frankfurt_am_Main.gpkg")
buildings_frankfurt= gpd.read_file(r"C:\Users\nb\OneDrive\Documents\Halle\Hackathon local data\4 Buildings\Buildings_Frankfurt_am_Main.gpkg")

In [1018]:
#satelite_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\Neighborhoods_Frankfurt_am_Main.gpkg")

In [1109]:
satelite_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\Neighborhoods_Frankfurt_am_Main_3.gpkg")

In [1110]:
dist_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\ffm_dist.gpkg")

In [1111]:
socio_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\Halle\Hackathon local data\zensus_ffm.gpkg")

In [1112]:
socio_data = socio_data.groupby('Neighborhood_FID').first().reset_index()
socio_data = socio_data.set_index('Neighborhood_FID')
dist_data = dist_data.set_index('Neighborhood_FID')
satelite_data = satelite_data.set_index('Neighborhood_FID')
neighborhoods_frankfurt =neighborhoods_frankfurt.set_index('Neighborhood_FID')

In [1113]:
satelite_data = satelite_data.set_index('Neighborhood_FID')

KeyError: "None of ['Neighborhood_FID'] are in the columns"

In [1114]:
sat_col = list(satelite_data)

In [1115]:
sat_col[7:]

['NUM_subway',
 'NUM_railway',
 'NUM_shops',
 'NUM_hospitals',
 'NUM_schools',
 'NUM_parks',
 'affected_07',
 'affected_25',
 'geometry']

In [1116]:
socio_col = list(socio_data)

In [1117]:
socio_col[-9:]

['area',
 'pop_nbh',
 'gerborn_tot_nbh',
 'mig_share_nbh',
 'pop_density',
 'hh_child_couple_nbh',
 'hh_child_single_nbh',
 'hh_child_single_share_nbh',
 'geometry']

# Construct Variables - stored in df with neighborhoods in the rows
interesting: 
neighborhoods_frankfurt['Area_Types'] - create dummies whether mixed, residential, commercial...
buildings_frankfurt['Building_Type'] - residential apartments share, office buildings, indicator whether in 75th percentile in terms of amount of ofice buildings, number of sports_centers, 
satelite_data['treecover']
satelite_data['builtup']
satelite_data['shrubland']
satelite_data['grassland']

### satelite data and neighborhoods are already on neighborhood level, buildings is not. 

In [1118]:
satelite_data.head()

,Neighborhood_Name,City_Code,City_Name,treecover,shrubland,grassland,builtup,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,affected_07,affected_25,geometry
Neighborhood_FID,,,,,,,,,,,,,,,,
1,Altstadt,06412000,Frankfurt am Main,0.133689,0.0,0.009084,0.729463,0.0,0.0,4.0,0.0,0.0,5.0,NaN,NaN,"MULTIPOLYGON (((4226384.447 3000227.322, 42263..."
2,Innenstadt,06412000,Frankfurt am Main,0.249146,0.0,0.008905,0.699297,7.0,3.0,9.0,1.0,3.0,18.0,NaN,NaN,"MULTIPOLYGON (((4227082.463 3001711.312, 42272..."
3,Bahnhofsviertel,06412000,Frankfurt am Main,0.084468,0.0,0.006073,0.840633,0.0,0.0,13.0,0.0,2.0,6.0,NaN,NaN,"MULTIPOLYGON (((4225284.051 3000486.630, 42252..."
4,Westend-Süd,06412000,Frankfurt am Main,0.308698,0.0,0.025584,0.658058,2.0,2.0,3.0,0.0,7.0,30.0,NaN,NaN,"MULTIPOLYGON (((4225219.883 3002076.156, 42252..."
5,Westend-Nord,06412000,Frankfurt am Main,0.478802,0.0,0.095331,0.420782,2.0,0.0,6.0,0.0,7.0,15.0,NaN,NaN,"MULTIPOLYGON (((4225993.384 3003603.082, 42260..."


In [1121]:
neigh_level_vars = gpd.GeoDataFrame()
neigh_level_vars['Land_Value']= neighborhoods_frankfurt['Land_Value']
neigh_level_vars['treecover']= satelite_data['treecover']
neigh_level_vars['builtup']= satelite_data['builtup']
neigh_level_vars['shrubland']= satelite_data['shrubland']
neigh_level_vars['grassland']= satelite_data['grassland']
neigh_level_vars['affected_07']= satelite_data['affected_07']
neigh_level_vars['affected_25']= satelite_data['affected_25']
neigh_level_vars[sat_col[7:]]=satelite_data[sat_col[7:]]
neigh_level_vars[socio_col[-9:]]=socio_data[socio_col[-9:]]
neigh_level_vars['dist']= dist_data['dist']


In [1122]:
neigh_level_vars= neigh_level_vars[:-1]

In [1123]:
neigh_level_vars.isnull().sum()

Land_Value                    0
treecover                     0
builtup                       0
shrubland                     0
grassland                     0
affected_07                  29
affected_25                  41
NUM_subway                    0
NUM_railway                   0
NUM_shops                     0
NUM_hospitals                 0
NUM_schools                   0
NUM_parks                     0
geometry                      0
area                          0
pop_nbh                       0
gerborn_tot_nbh               0
mig_share_nbh                 0
pop_density                   0
hh_child_couple_nbh           0
hh_child_single_nbh           0
hh_child_single_share_nbh     1
dist                          0
dtype: int64

In [1124]:
satelite_data[satelite_data['builtup'].isnull()]

,Neighborhood_Name,City_Code,City_Name,treecover,shrubland,grassland,builtup,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,affected_07,affected_25,geometry
Neighborhood_FID,,,,,,,,,,,,,,,,


In [1125]:
dist_data.head()

,Neighborhood_Name,City_Code,City_Name,dist,geometry
Neighborhood_FID,,,,,
1,Altstadt,06412000,Frankfurt am Main,0.00000,"MULTIPOLYGON (((4226384.44674 3000227.32158, 4..."
2,Innenstadt,06412000,Frankfurt am Main,0.00000,"MULTIPOLYGON (((4227082.46278 3001711.31224, 4..."
3,Bahnhofsviertel,06412000,Frankfurt am Main,0.00000,"MULTIPOLYGON (((4225284.05145 3000486.62977, 4..."
4,Westend-Süd,06412000,Frankfurt am Main,0.00000,"MULTIPOLYGON (((4225219.88286 3002076.15554, 4..."
5,Westend-Nord,06412000,Frankfurt am Main,304.48417,"MULTIPOLYGON (((4225993.38385 3003603.08203, 4..."


### normalize variables

In [1126]:
neigh_level_vars['area']= neigh_level_vars['area']*10000

In [1127]:
for element in ['NUM_shops', 'NUM_railway', 'NUM_subway', 'NUM_hospitals', 'NUM_schools', 'NUM_parks']: 
    
    neigh_level_vars[element]= neigh_level_vars[element]/neigh_level_vars['area'].values

### type of buildings - shares


### residential and non-residential

In [1128]:
buildings_frankfurt['resid']= np.where(
    buildings_frankfurt['Building_TypeGen'] == 'residential', 1, np.where(
    buildings_frankfurt['Building_TypeGen'] == 'non-residential' , 0, 0)) 

In [1129]:
buildings_frankfurt['nresid']= np.where(
    buildings_frankfurt['Building_TypeGen'] == 'non-residential', 1, np.where(
    buildings_frankfurt['Building_TypeGen'] == 'residential' , 0, 0)) 

In [1130]:
neighb_aggr2 = buildings_frankfurt.pivot_table(values = 'resid', index = 'Neighborhood_FID', columns = 'Build_ID')
neighb_aggr2['resid']=neighb_aggr2.sum(axis=1)
neigh_level_vars['resid'] = neighb_aggr2['resid']

In [1131]:
neighb_aggr3 = buildings_frankfurt.pivot_table(values = 'nresid', index = 'Neighborhood_FID', columns = 'Build_ID')
neighb_aggr3['nresid']=neighb_aggr3.sum(axis=1)
neigh_level_vars['nresid'] = neighb_aggr3['nresid']

In [1132]:
neigh_level_vars['build_count']= neigh_level_vars['resid'] + neigh_level_vars['nresid']
neigh_level_vars['resid_share']= neigh_level_vars['resid']/neigh_level_vars['build_count'].values
#.div(neigh_level_vars['build_count'].iloc[0], axis='rows')
#neigh_level_vars['resid'].groupby('Neighborhood_FID').divide(neigh_level_vars['build_count'])

### average building height


In [1133]:
neighb_aggr = buildings_frankfurt.pivot_table(values = 'Building_Height', index = 'Neighborhood_FID', columns = 'Build_ID')
neighb_aggr['avg_height']=neighb_aggr.sum(axis=1)
neigh_level_vars['avg_height']= neighb_aggr['avg_height']

### number of houses

also interesting: the share of houses vs. apartments - number of people per square meter

In [1134]:
neighb_aggr = gpd.GeoDataFrame()
neighb_aggr[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = buildings_frankfurt[buildings_frankfurt['Building_Type']=='house'].groupby('Neighborhood_FID').count()
neigh_level_vars['house_count']= neighb_aggr['1']

In [1135]:
neighb_aggr = gpd.GeoDataFrame()
neighb_aggr[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = buildings_frankfurt[buildings_frankfurt['Building_Type']=='apartments'].groupby('Neighborhood_FID').count()
neigh_level_vars['apartment_count']= neighb_aggr['1']

In [1136]:
neighb_aggr = gpd.GeoDataFrame()
neighb_aggr[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = buildings_frankfurt[buildings_frankfurt['Building_Type']=='office'].groupby('Neighborhood_FID').count()
neighb_aggr_final['office_count']= neighb_aggr['1']

## type of Area

In [1137]:
test = gpd.Geo.DataFrame
test = neighborhoods_frankfurt['Area_Types'].str.split(pat='_',expand=True)

AttributeError: module 'geopandas' has no attribute 'Geo'

In [1138]:
neigh_level_vars[['areatype_1', 'areatype_2', 'areatype_3']]= test[[0, 1, 2]]

In [1139]:
neigh_level_vars['areatype_1'].value_counts()

areatype_1
AgriculturalArea          16
CommercialBuildingArea    15
MixedUseArea              13
OtherArea                  1
Name: count, dtype: int64

In [1140]:
neigh_level_vars['areatype_agri'] = np.where(neigh_level_vars['areatype_1']=='AgriculturalArea', 1, 0)
neigh_level_vars['areatype_commer'] = np.where(neigh_level_vars['areatype_1']=='CommercialBuildingArea', 1, 0)
neigh_level_vars['areatype_mixed'] = np.where(neigh_level_vars['areatype_1']=='MixedUseArea', 1, 0)

In [1141]:
neigh_level_vars = neigh_level_vars.drop([ 'areatype_1','areatype_2', 'areatype_3', 'resid', 'nresid'], axis=1)

In [1142]:
neigh_level_vars['house_count'].fillna(neigh_level_vars['house_count'].mean(), inplace=True)
neigh_level_vars['apartment_count'].fillna(neigh_level_vars['apartment_count'].mean(), inplace=True)

neigh_level_vars['hh_child_single_share_nbh'].fillna(neigh_level_vars.iloc[[13, 17],18].mean(), inplace=True)
neigh_level_vars = neigh_level_vars.drop('geometry', axis=1)

In [1143]:
neigh_level_vars['affected_07'].fillna(0, inplace=True)
neigh_level_vars['affected_25'].fillna(0, inplace=True)

# Regression

In [1144]:
#x= np.asarray(neigh_level_vars[:,1:])
#y= neigh_level_vars[:,0]
 
#x = sm.add_constant(x)
#.reshape((-1, 1))
#model2 = sm.OLS(y, x)
#model2 = sm.OLS(np.asarray(y).reshape((-1, 1)), x)


In [1145]:
#model2 = sm.OLS(np.asarray(withprice[dependent]).reshape((-1, 1)), x)

In [1146]:
#fit = model2.fit()
#model.fit(x,np.asarray(withprice[dependent]).reshape((-1, 1)))
#withprice['price_prediction'] = fit.predict(withprice[predictors])
#results = model2.fit()

# Lasso

In [1147]:
neigh_level_vars.isnull().sum()

Land_Value                   0
treecover                    0
builtup                      0
shrubland                    0
grassland                    0
affected_07                  0
affected_25                  0
NUM_subway                   0
NUM_railway                  0
NUM_shops                    0
NUM_hospitals                0
NUM_schools                  0
NUM_parks                    0
area                         0
pop_nbh                      0
gerborn_tot_nbh              0
mig_share_nbh                0
pop_density                  0
hh_child_couple_nbh          0
hh_child_single_nbh          0
hh_child_single_share_nbh    0
dist                         0
build_count                  0
resid_share                  0
avg_height                   0
house_count                  0
apartment_count              0
areatype_agri                0
areatype_commer              0
areatype_mixed               0
dtype: int64

In [1148]:
model_LASSO = Lasso(alpha=0.5)
#model = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# force scores to be positive
#scores = absolute(scores)
#print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

#withprice2= withprice[['office_perc75', 'resid_share', 'Land_Value']]
data = neigh_level_vars.values


In [1149]:
X_LASSO, y_LASSO = data[:30,1: ], data[:30, 0]
# evaluate model
scores = cross_val_score(model_LASSO, X_LASSO, y_LASSO, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
model_LASSO.fit(X_LASSO,y_LASSO)

X_LASSO_test, y_LASSO_test = data[31:,1: ], data[31:, 0]

C:\Users\nb\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.003e+07, tolerance: 2.166e+04
  model = cd_fast.enet_coordinate_descent(


In [1150]:
results_prices_train= gpd.GeoDataFrame()
results_prices_train['Lasso_pred']= model_LASSO.predict(X_LASSO)

In [1151]:
results_prices_test = gpd.GeoDataFrame()
results_prices_test['Lasso_pred']= model_LASSO.predict(X_LASSO_test)

In [1152]:
results_prices_test

,Lasso_pred
0,-1130.380766
1,1228.185334
2,1210.801811
3,-4391.697370
4,225.056754
5,-9510.275084
6,-17073.673449
7,-6160.128975
8,-4602.293538
9,1473.140826


In [1153]:
results_prices_test

,Lasso_pred
0,-1130.380766
1,1228.185334
2,1210.801811
3,-4391.697370
4,225.056754
5,-9510.275084
6,-17073.673449
7,-6160.128975
8,-4602.293538
9,1473.140826


# OLS

In [1249]:
coefficients = list(neigh_level_vars)
coefficients= coefficients[:-1]
X = neigh_level_vars[coefficients]
y = neigh_level_vars['Land_Value']

In [1250]:
print(coefficients)

['Land_Value', 'treecover', 'builtup', 'shrubland', 'grassland', 'affected_07', 'affected_25', 'NUM_subway', 'NUM_railway', 'NUM_shops', 'NUM_hospitals', 'NUM_schools', 'NUM_parks', 'area', 'pop_nbh', 'gerborn_tot_nbh', 'mig_share_nbh', 'pop_density', 'hh_child_couple_nbh', 'hh_child_single_nbh', 'hh_child_single_share_nbh', 'dist', 'build_count', 'resid_share', 'avg_height', 'house_count', 'apartment_count', 'areatype_agri', 'areatype_commer', 'areatype_mixed']


In [1251]:
# Evaluate Model

#X_OLS, y_OLS = data[:30,1: ], data[:30, 0]
# evaluate model
#scores = cross_val_score(model_LASSO, X_LASSO, y_LASSO, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#model_LASSO.fit(X_LASSO,y_LASSO)
#X_LASSO_test, y_LASSO_test = data[31:,1: ], data[31:, 0]

In [1252]:
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
regr = linear_model.LinearRegression()
regr.fit(X, y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

Intercept: 
 -4.092726157978177e-12
Coefficients: 
 [ 1.00000000e+00 -8.51000922e-13  2.81243595e-12  1.06385622e-08
  4.30243193e-12 -7.53894685e-14 -4.41288836e-14 -1.10423991e-09
 -9.38309738e-09 -2.52348806e-10  2.94174546e-09 -2.62678458e-09
 -1.93705162e-10  6.17644667e-17  1.56190457e-16 -2.60007799e-16
 -3.12213677e-12  1.97183178e-16 -1.03840644e-15  3.04272430e-15
 -3.05593011e-15 -1.84466359e-16  4.73592679e-16  1.38213774e-12
  2.47493247e-16 -4.48315042e-16 -1.29845318e-15 -7.12019969e-13
 -9.50671666e-13 -5.80440343e-14]


In [1257]:
coefs=(pd.DataFrame({'coef':regr.coef_, 'category':X.columns})
.sort_values(by = 'coef', ascending = False)
.set_index('category'))

print(coefs)

                                   coef
category                               
Land_Value                 1.000000e+00
shrubland                  1.063856e-08
NUM_hospitals              2.941745e-09
grassland                  4.302432e-12
builtup                    2.812436e-12
resid_share                1.382138e-12
hh_child_single_nbh        3.042724e-15
build_count                4.735927e-16
avg_height                 2.474932e-16
pop_density                1.971832e-16
pop_nbh                    1.561905e-16
area                       6.176447e-17
dist                      -1.844664e-16
gerborn_tot_nbh           -2.600078e-16
house_count               -4.483150e-16
hh_child_couple_nbh       -1.038406e-15
apartment_count           -1.298453e-15
hh_child_single_share_nbh -3.055930e-15
affected_25               -4.412888e-14
areatype_mixed            -5.804403e-14
affected_07               -7.538947e-14
areatype_agri             -7.120200e-13
treecover                 -8.510009e-13


In [1259]:
coefs.head()

,coef
category,
Land_Value,1.000000e+00
shrubland,1.063856e-08
NUM_hospitals,2.941745e-09
grassland,4.302432e-12
builtup,2.812436e-12


In [969]:
results_prices['OLS']= predictions_OLS

In [982]:

resetindex= gpd.GeoDataFrame()
resetindex['Land_Value']=neigh_level_vars['Land_Value']
#.reset_index()

In [986]:
resetindex = resetindex.reset_index()

In [987]:
results_prices['True']= resetindex['Land_Value']

In [988]:
results_prices

,Lasso_pred,OLS,True
0,6335.683149,6606.406871,6998.5
1,7294.687769,10394.090648,11756.9
2,6426.564044,4914.427888,5312.2
3,5845.048125,7486.571407,9779.6
4,4574.945292,4987.302562,4236.2
5,6456.720683,6308.279985,6407.0
6,6003.036547,5697.746134,5529.3
7,3129.823408,4568.469008,4334.1
8,4785.035081,3560.728664,3302.5
9,5384.525091,5833.157652,3374.7


In [1157]:
neigh_level_vars_streamlite=gpd.GeoDataFrame()
neigh_level_vars_streamlite = neigh_level_vars
#neigh_level_vars_streamlite.drop(neigh_level_vars_streamlite['Land_Value'], axis=1)

In [1158]:
neigh_level_vars_streamlite

,Land_Value,treecover,builtup,shrubland,grassland,affected_07,affected_25,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,area,pop_nbh,gerborn_tot_nbh,mig_share_nbh,pop_density,hh_child_couple_nbh,hh_child_single_nbh,hh_child_single_share_nbh,dist,build_count,resid_share,avg_height,house_count,apartment_count,areatype_agri,areatype_commer,areatype_mixed,geometry
Neighborhood_FID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,6998.5,0.133689,0.729463,0.000000,0.009084,0.0,0.0,0.000000,0.000000,0.000786,0.000000,0.000000,0.000982,5091.031541,2818,1666,0.408801,5535.224006,206,93,0.311037,0.000000,72.0,0.791667,954.5,4.000000,48.000000,0,0,1,"MULTIPOLYGON (((4226384.44674 3000227.32158, 4..."
2,11756.9,0.249146,0.699297,0.000000,0.008905,0.0,0.0,0.000467,0.000200,0.000601,0.000067,0.000200,0.001202,14976.591516,5869,3081,0.475038,3918.782183,315,138,0.304636,0.000000,194.0,0.350515,2907.1,3.000000,64.000000,0,0,1,"MULTIPOLYGON (((4227082.46278 3001711.31224, 4..."
3,5312.2,0.084468,0.840633,0.000000,0.006073,0.0,0.0,0.000000,0.000000,0.002384,0.000000,0.000367,0.001100,5452.697722,2268,1115,0.508377,4159.409004,108,27,0.200000,0.000000,130.0,0.400000,1693.5,1.000000,50.000000,0,0,1,"MULTIPOLYGON (((4225284.05145 3000486.62977, 4..."
4,9779.6,0.308698,0.658058,0.000000,0.025584,0.0,0.0,0.000080,0.000080,0.000120,0.000000,0.000279,0.001196,25074.682771,15240,10577,0.305971,6077.843592,1348,351,0.206592,0.000000,480.0,0.575000,4654.8,1.000000,263.000000,0,1,0,"MULTIPOLYGON (((4225219.88286 3002076.15554, 4..."
5,4236.2,0.478802,0.420782,0.000000,0.095331,0.0,0.0,0.000122,0.000000,0.000366,0.000000,0.000427,0.000915,16389.890354,6267,4259,0.320408,3823.698551,625,177,0.220698,304.484170,647.0,0.761978,5052.0,17.000000,421.000000,0,0,1,"MULTIPOLYGON (((4225993.38385 3003603.08203, 4..."
6,6407.0,0.422453,0.539943,0.000000,0.036121,0.0,0.0,0.000193,0.000000,0.000353,0.000064,0.000353,0.000675,31132.536753,24706,18798,0.239132,7935.749083,2075,778,0.272695,0.000000,1319.0,0.822593,7173.5,11.000000,1060.000000,0,0,1,"MULTIPOLYGON (((4227858.46392 3003565.57485, 4..."
7,5529.3,0.447289,0.524233,0.000000,0.026845,0.0,0.0,0.000130,0.000000,0.000325,0.000000,0.000455,0.001430,15381.550793,16946,12891,0.239290,11017.094589,1351,610,0.311066,0.000000,1403.0,0.908767,3291.5,143.000000,1117.000000,0,1,0,"MULTIPOLYGON (((4227867.21204 3001590.05997, 4..."
8,4334.1,0.196036,0.656307,0.000000,0.049009,1.0,0.0,0.000090,0.000036,0.000233,0.000036,0.000143,0.000412,55848.432463,22410,14706,0.343775,4012.646195,1725,702,0.289246,0.000000,2163.0,0.714748,10443.5,38.000000,1476.000000,0,1,0,"MULTIPOLYGON (((4231428.84153 3001758.58257, 4..."
9,3302.5,0.447060,0.476517,0.000000,0.065875,0.0,0.0,0.000107,0.000000,0.000429,0.000071,0.000250,0.000322,27978.401596,25316,18316,0.276505,9048.408256,1974,945,0.323741,864.461323,2969.0,0.807343,16737.5,362.000000,2028.000000,0,0,1,"MULTIPOLYGON (((4228434.89586 3001994.71993, 4..."


In [1171]:
neigh_level_vars_streamlite.drop('Land_Value', axis=1)

,treecover,builtup,shrubland,grassland,affected_07,affected_25,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,area,pop_nbh,gerborn_tot_nbh,mig_share_nbh,pop_density,hh_child_couple_nbh,hh_child_single_nbh,hh_child_single_share_nbh,dist,build_count,resid_share,avg_height,house_count,apartment_count,areatype_agri,areatype_commer,areatype_mixed,geometry
Neighborhood_FID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.133689,0.729463,0.000000,0.009084,0.0,0.0,0.000000,0.000000,0.000786,0.000000,0.000000,0.000982,5091.031541,2818,1666,0.408801,5535.224006,206,93,0.311037,0.000000,72.0,0.791667,954.5,4.000000,48.000000,0,0,1,"MULTIPOLYGON (((4226384.44674 3000227.32158, 4..."
2,0.249146,0.699297,0.000000,0.008905,0.0,0.0,0.000467,0.000200,0.000601,0.000067,0.000200,0.001202,14976.591516,5869,3081,0.475038,3918.782183,315,138,0.304636,0.000000,194.0,0.350515,2907.1,3.000000,64.000000,0,0,1,"MULTIPOLYGON (((4227082.46278 3001711.31224, 4..."
3,0.084468,0.840633,0.000000,0.006073,0.0,0.0,0.000000,0.000000,0.002384,0.000000,0.000367,0.001100,5452.697722,2268,1115,0.508377,4159.409004,108,27,0.200000,0.000000,130.0,0.400000,1693.5,1.000000,50.000000,0,0,1,"MULTIPOLYGON (((4225284.05145 3000486.62977, 4..."
4,0.308698,0.658058,0.000000,0.025584,0.0,0.0,0.000080,0.000080,0.000120,0.000000,0.000279,0.001196,25074.682771,15240,10577,0.305971,6077.843592,1348,351,0.206592,0.000000,480.0,0.575000,4654.8,1.000000,263.000000,0,1,0,"MULTIPOLYGON (((4225219.88286 3002076.15554, 4..."
5,0.478802,0.420782,0.000000,0.095331,0.0,0.0,0.000122,0.000000,0.000366,0.000000,0.000427,0.000915,16389.890354,6267,4259,0.320408,3823.698551,625,177,0.220698,304.484170,647.0,0.761978,5052.0,17.000000,421.000000,0,0,1,"MULTIPOLYGON (((4225993.38385 3003603.08203, 4..."
6,0.422453,0.539943,0.000000,0.036121,0.0,0.0,0.000193,0.000000,0.000353,0.000064,0.000353,0.000675,31132.536753,24706,18798,0.239132,7935.749083,2075,778,0.272695,0.000000,1319.0,0.822593,7173.5,11.000000,1060.000000,0,0,1,"MULTIPOLYGON (((4227858.46392 3003565.57485, 4..."
7,0.447289,0.524233,0.000000,0.026845,0.0,0.0,0.000130,0.000000,0.000325,0.000000,0.000455,0.001430,15381.550793,16946,12891,0.239290,11017.094589,1351,610,0.311066,0.000000,1403.0,0.908767,3291.5,143.000000,1117.000000,0,1,0,"MULTIPOLYGON (((4227867.21204 3001590.05997, 4..."
8,0.196036,0.656307,0.000000,0.049009,1.0,0.0,0.000090,0.000036,0.000233,0.000036,0.000143,0.000412,55848.432463,22410,14706,0.343775,4012.646195,1725,702,0.289246,0.000000,2163.0,0.714748,10443.5,38.000000,1476.000000,0,1,0,"MULTIPOLYGON (((4231428.84153 3001758.58257, 4..."
9,0.447060,0.476517,0.000000,0.065875,0.0,0.0,0.000107,0.000000,0.000429,0.000071,0.000250,0.000322,27978.401596,25316,18316,0.276505,9048.408256,1974,945,0.323741,864.461323,2969.0,0.807343,16737.5,362.000000,2028.000000,0,0,1,"MULTIPOLYGON (((4228434.89586 3001994.71993, 4..."


In [1165]:
neigh_level_vars_streamlite.to_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\data_for_streamlit.gpkg", driver="GPKG")

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [1170]:
neigh_level_vars_streamlite.to_csv('for_streamlit.csv', sep='\t')

In [1104]:
# CRS looks like a UTM. Enter EPSG code of UTM here. For example 32637
#crs_code = 32637

#neigh_level_vars_streamlite['geometry'] = GeoSeries.from_wkt(neigh_level_vars_streamlite['geometry'])
#neigh_level_vars_streamlite = GeoDataFrame(neigh_level_vars_streamlite, crs=crs_code)

#grid_up2geo.to_file('grid_up2geo.gpkg', driver='GPKG')

TypeError: Expected bytes or string, got MultiPolygon

In [1098]:
neigh_level_vars_streamlite.to_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\data_for_streamlite.gpkg", driver="GPKG")

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [1172]:
neighborhoods_frankfurt

,Land_Value,Area_Types,Area_Count,City_Name,geometry
Neighborhood_FID,,,,,
1,6998.5,MixedUseArea_Residential,7,Frankfurt am Main,"MULTIPOLYGON (((4226384.447 3000227.322, 42263..."
2,11756.9,MixedUseArea_Residential_SpecialUseArea,21,Frankfurt am Main,"MULTIPOLYGON (((4227082.463 3001711.312, 42272..."
3,5312.2,MixedUseArea_Residential,9,Frankfurt am Main,"MULTIPOLYGON (((4225284.051 3000486.630, 42252..."
4,9779.6,CommercialBuildingArea_MixedUseArea_Residentia...,20,Frankfurt am Main,"MULTIPOLYGON (((4225219.883 3002076.156, 42252..."
5,4236.2,MixedUseArea_OtherArea_Residential_SpecialUseArea,20,Frankfurt am Main,"MULTIPOLYGON (((4225993.384 3003603.082, 42260..."
6,6407.0,MixedUseArea_Residential_SpecialUseArea,17,Frankfurt am Main,"MULTIPOLYGON (((4227858.464 3003565.575, 42278..."
7,5529.3,CommercialBuildingArea_MixedUseArea_OtherArea_...,17,Frankfurt am Main,"MULTIPOLYGON (((4227867.212 3001590.060, 42277..."
8,4334.1,CommercialBuildingArea_MixedUseArea_Residential,20,Frankfurt am Main,"MULTIPOLYGON (((4231428.842 3001758.583, 42314..."
9,3302.5,MixedUseArea_OtherArea_Residential,21,Frankfurt am Main,"MULTIPOLYGON (((4228434.896 3001994.720, 42284..."


In [1177]:
neigh_level_vars_streamlite_geo = neighborhoods_frankfurt.merge(neigh_level_vars_streamlite, on = index, how ="inner" )

NameError: name 'index' is not defined

In [1180]:
neigh_level_vars_streamlite.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 1 to 45
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Land_Value                 45 non-null     float64 
 1   treecover                  45 non-null     float64 
 2   builtup                    45 non-null     float64 
 3   shrubland                  45 non-null     float64 
 4   grassland                  45 non-null     float64 
 5   affected_07                45 non-null     float64 
 6   affected_25                45 non-null     float64 
 7   NUM_subway                 45 non-null     float64 
 8   NUM_railway                45 non-null     float64 
 9   NUM_shops                  45 non-null     float64 
 10  NUM_hospitals              45 non-null     float64 
 11  NUM_schools                45 non-null     float64 
 12  NUM_parks                  45 non-null     float64 
 13  area                       45 non-null    

In [1182]:
neighborhoods_frankfurt.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 46 entries, 1 to 46
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Land_Value  46 non-null     float64 
 1   Area_Types  46 non-null     object  
 2   Area_Count  46 non-null     int64   
 3   City_Name   46 non-null     object  
 4   geometry    46 non-null     geometry
dtypes: float64(1), geometry(1), int64(1), object(2)
memory usage: 2.2+ KB


In [1184]:
gdf= gpd.GeoDataFrame(neigh_level_vars_streamlite)

In [1190]:
gdf.drop('geometry', axis=1)

,Land_Value,treecover,builtup,shrubland,grassland,affected_07,affected_25,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,area,pop_nbh,gerborn_tot_nbh,mig_share_nbh,pop_density,hh_child_couple_nbh,hh_child_single_nbh,hh_child_single_share_nbh,dist,build_count,resid_share,avg_height,house_count,apartment_count,areatype_agri,areatype_commer,areatype_mixed
Neighborhood_FID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,6998.5,0.133689,0.729463,0.000000,0.009084,0.0,0.0,0.000000,0.000000,0.000786,0.000000,0.000000,0.000982,5091.031541,2818,1666,0.408801,5535.224006,206,93,0.311037,0.000000,72.0,0.791667,954.5,4.000000,48.000000,0,0,1
2,11756.9,0.249146,0.699297,0.000000,0.008905,0.0,0.0,0.000467,0.000200,0.000601,0.000067,0.000200,0.001202,14976.591516,5869,3081,0.475038,3918.782183,315,138,0.304636,0.000000,194.0,0.350515,2907.1,3.000000,64.000000,0,0,1
3,5312.2,0.084468,0.840633,0.000000,0.006073,0.0,0.0,0.000000,0.000000,0.002384,0.000000,0.000367,0.001100,5452.697722,2268,1115,0.508377,4159.409004,108,27,0.200000,0.000000,130.0,0.400000,1693.5,1.000000,50.000000,0,0,1
4,9779.6,0.308698,0.658058,0.000000,0.025584,0.0,0.0,0.000080,0.000080,0.000120,0.000000,0.000279,0.001196,25074.682771,15240,10577,0.305971,6077.843592,1348,351,0.206592,0.000000,480.0,0.575000,4654.8,1.000000,263.000000,0,1,0
5,4236.2,0.478802,0.420782,0.000000,0.095331,0.0,0.0,0.000122,0.000000,0.000366,0.000000,0.000427,0.000915,16389.890354,6267,4259,0.320408,3823.698551,625,177,0.220698,304.484170,647.0,0.761978,5052.0,17.000000,421.000000,0,0,1
6,6407.0,0.422453,0.539943,0.000000,0.036121,0.0,0.0,0.000193,0.000000,0.000353,0.000064,0.000353,0.000675,31132.536753,24706,18798,0.239132,7935.749083,2075,778,0.272695,0.000000,1319.0,0.822593,7173.5,11.000000,1060.000000,0,0,1
7,5529.3,0.447289,0.524233,0.000000,0.026845,0.0,0.0,0.000130,0.000000,0.000325,0.000000,0.000455,0.001430,15381.550793,16946,12891,0.239290,11017.094589,1351,610,0.311066,0.000000,1403.0,0.908767,3291.5,143.000000,1117.000000,0,1,0
8,4334.1,0.196036,0.656307,0.000000,0.049009,1.0,0.0,0.000090,0.000036,0.000233,0.000036,0.000143,0.000412,55848.432463,22410,14706,0.343775,4012.646195,1725,702,0.289246,0.000000,2163.0,0.714748,10443.5,38.000000,1476.000000,0,1,0
9,3302.5,0.447060,0.476517,0.000000,0.065875,0.0,0.0,0.000107,0.000000,0.000429,0.000071,0.000250,0.000322,27978.401596,25316,18316,0.276505,9048.408256,1974,945,0.323741,864.461323,2969.0,0.807343,16737.5,362.000000,2028.000000,0,0,1


In [1191]:
satelite_data = gpd.read_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\Neighborhoods_Frankfurt_am_Main_3.gpkg")

In [1194]:
satelite_data.head()

,Neighborhood_FID,Neighborhood_Name,City_Code,City_Name,treecover,shrubland,grassland,builtup,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,affected_07,affected_25,geometry
0,1,Altstadt,06412000,Frankfurt am Main,0.133689,0.0,0.009084,0.729463,0.0,0.0,4.0,0.0,0.0,5.0,NaN,NaN,"MULTIPOLYGON (((4226384.447 3000227.322, 42263..."
1,2,Innenstadt,06412000,Frankfurt am Main,0.249146,0.0,0.008905,0.699297,7.0,3.0,9.0,1.0,3.0,18.0,NaN,NaN,"MULTIPOLYGON (((4227082.463 3001711.312, 42272..."
2,3,Bahnhofsviertel,06412000,Frankfurt am Main,0.084468,0.0,0.006073,0.840633,0.0,0.0,13.0,0.0,2.0,6.0,NaN,NaN,"MULTIPOLYGON (((4225284.051 3000486.630, 42252..."
3,4,Westend-Süd,06412000,Frankfurt am Main,0.308698,0.0,0.025584,0.658058,2.0,2.0,3.0,0.0,7.0,30.0,NaN,NaN,"MULTIPOLYGON (((4225219.883 3002076.156, 42252..."
4,5,Westend-Nord,06412000,Frankfurt am Main,0.478802,0.0,0.095331,0.420782,2.0,0.0,6.0,0.0,7.0,15.0,NaN,NaN,"MULTIPOLYGON (((4225993.384 3003603.082, 42260..."


In [1192]:
gdf['geometry']=satelite_data['geometry']

In [1195]:
gdf['Neighborhood']=satelite_data['Neighborhood_Name']

In [1230]:
gdf['flugschneise2']= gdf['affected_07']+gdf['affected_25']

In [1231]:
gdf['flugschneise2'][(gdf['affected_07']==1) & (gdf['affected_25']==1)] = 1
gdf['flugschneise2'][(gdf['affected_07']==1) | (gdf['affected_25']==1)] = 1
gdf['flugschneise2'][(gdf['affected_07']!=1) & (gdf['affected_25']!=1)] = 0

#gdf['flugschneise'].map({2: 1, 1: 1, 0:0})

C:\Users\nb\AppData\Local\Temp\ipykernel_7020\3115916223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['flugschneise2'][(gdf['affected_07']==1) & (gdf['affected_25']==1)] = 1
C:\Users\nb\AppData\Local\Temp\ipykernel_7020\3115916223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['flugschneise2'][(gdf['affected_07']==1) | (gdf['affected_25']==1)] = 1
C:\Users\nb\AppData\Local\Temp\ipykernel_7020\3115916223.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [1197]:
gdf.to_file(r"C:\Users\nb\OneDrive\Documents\GitHub\vier_gewinnt\data\streamlit2.gpkg", driver="GPKG" )

In [1234]:
gdf.drop(['affected_07', 'affected_25', 'flugschneise'], axis=1)
gdf['flugschneise']= gdf['flugschneise2']

In [1235]:
gdf.drop('flugschneise2', axis=1)

,Land_Value,treecover,builtup,shrubland,grassland,affected_07,affected_25,NUM_subway,NUM_railway,NUM_shops,NUM_hospitals,NUM_schools,NUM_parks,area,pop_nbh,gerborn_tot_nbh,mig_share_nbh,pop_density,hh_child_couple_nbh,hh_child_single_nbh,hh_child_single_share_nbh,dist,build_count,resid_share,avg_height,house_count,apartment_count,areatype_agri,areatype_commer,areatype_mixed,geometry,Neighborhood,flugschneise
Neighborhood_FID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,6998.5,0.133689,0.729463,0.000000,0.009084,0.0,0.0,0.000000,0.000000,0.000786,0.000000,0.000000,0.000982,5091.031541,2818,1666,0.408801,5535.224006,206,93,0.311037,0.000000,72.0,0.791667,954.5,4.000000,48.000000,0,0,1,"MULTIPOLYGON (((4227082.463 3001711.312, 42272...",Innenstadt,0.0
2,11756.9,0.249146,0.699297,0.000000,0.008905,0.0,0.0,0.000467,0.000200,0.000601,0.000067,0.000200,0.001202,14976.591516,5869,3081,0.475038,3918.782183,315,138,0.304636,0.000000,194.0,0.350515,2907.1,3.000000,64.000000,0,0,1,"MULTIPOLYGON (((4225284.051 3000486.630, 42252...",Bahnhofsviertel,0.0
3,5312.2,0.084468,0.840633,0.000000,0.006073,0.0,0.0,0.000000,0.000000,0.002384,0.000000,0.000367,0.001100,5452.697722,2268,1115,0.508377,4159.409004,108,27,0.200000,0.000000,130.0,0.400000,1693.5,1.000000,50.000000,0,0,1,"MULTIPOLYGON (((4225219.883 3002076.156, 42252...",Westend-Süd,0.0
4,9779.6,0.308698,0.658058,0.000000,0.025584,0.0,0.0,0.000080,0.000080,0.000120,0.000000,0.000279,0.001196,25074.682771,15240,10577,0.305971,6077.843592,1348,351,0.206592,0.000000,480.0,0.575000,4654.8,1.000000,263.000000,0,1,0,"MULTIPOLYGON (((4225993.384 3003603.082, 42260...",Westend-Nord,0.0
5,4236.2,0.478802,0.420782,0.000000,0.095331,0.0,0.0,0.000122,0.000000,0.000366,0.000000,0.000427,0.000915,16389.890354,6267,4259,0.320408,3823.698551,625,177,0.220698,304.484170,647.0,0.761978,5052.0,17.000000,421.000000,0,0,1,"MULTIPOLYGON (((4227858.464 3003565.575, 42278...",Nordend-West,0.0
6,6407.0,0.422453,0.539943,0.000000,0.036121,0.0,0.0,0.000193,0.000000,0.000353,0.000064,0.000353,0.000675,31132.536753,24706,18798,0.239132,7935.749083,2075,778,0.272695,0.000000,1319.0,0.822593,7173.5,11.000000,1060.000000,0,0,1,"MULTIPOLYGON (((4227867.212 3001590.060, 42277...",Nordend-Ost,0.0
7,5529.3,0.447289,0.524233,0.000000,0.026845,0.0,0.0,0.000130,0.000000,0.000325,0.000000,0.000455,0.001430,15381.550793,16946,12891,0.239290,11017.094589,1351,610,0.311066,0.000000,1403.0,0.908767,3291.5,143.000000,1117.000000,0,1,0,"MULTIPOLYGON (((4231428.842 3001758.583, 42314...",Ostend,0.0
8,4334.1,0.196036,0.656307,0.000000,0.049009,1.0,0.0,0.000090,0.000036,0.000233,0.000036,0.000143,0.000412,55848.432463,22410,14706,0.343775,4012.646195,1725,702,0.289246,0.000000,2163.0,0.714748,10443.5,38.000000,1476.000000,0,1,0,"MULTIPOLYGON (((4228434.896 3001994.720, 42284...",Bornheim,1.0
9,3302.5,0.447060,0.476517,0.000000,0.065875,0.0,0.0,0.000107,0.000000,0.000429,0.000071,0.000250,0.000322,27978.401596,25316,18316,0.276505,9048.408256,1974,945,0.323741,864.461323,2969.0,0.807343,16737.5,362.000000,2028.000000,0,0,1,"MULTIPOLYGON (((4222367.116 2998542.440, 42223...",Gutleutviertel,0.0


# Old

## first: connect datasets

### prices and buildings/neighb_aggr_final
I can join them based on Neighborhood_FID and if not enough Neighborhoof_Name

In [35]:
withprice = neighb_aggr_final.join(neighborhoods_frankfurt.set_index('Neighborhood_FID'))

In [36]:
predictors = ['resid_share', 'office_perc75', 'avg_height']
predictors2 = ['resid_share', 'office_perc75', 'avg_height']
dependent = 'Land_Value'
vars_with_nan ='office_perc75'
vars_without_nan = predictors2.remove(vars_with_nan)

## predict missings

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import statsmodels.api as sm

Dataset cannot include nan. can we predict them?


create new dataframe from which I can delete all values where office_perc75 is nan

In [38]:
df_for_predict = gpd.GeoDataFrame()
df_for_predict[predictors] = withprice[predictors]

In [39]:
# delete the variable with missings from df_for_predict
df_for_predict_sel= df_for_predict.drop(vars_with_nan , axis=1)
# drop observations (rows) with missings
df2 = df_for_predict.dropna(subset=vars_with_nan) 

In [40]:
# change x columns to arrays for fitting the model
a=np.asarray(df2.drop(vars_with_nan , axis=1))
#a=np.asarray(df2[['resid_share','avg_height']])


In [41]:
model_nan_office = LinearRegression(fit_intercept=True)
model_nan_office.fit(a, df2['office_perc75'])

LinearRegression()

### now predict the missing values of office_perc75 in withprice.

In [42]:
withprice['office_perc75_pred']= model_nan_office.predict(df_for_predict_sel)
#withprice['office_perc75_pred']= model_nan_office.predict(df[['resid_share','avg_height']])
#df_for_predict.drop(vars_with_nan , axis=1)

C:\Users\nb\anaconda3\envs\ifohack_spatial_py310\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


### now I have the predictions for office_perc75 and can replace nan with the prediction

In [43]:
withprice['office_perc75'].fillna(withprice['office_perc75_pred'], inplace=True)

## predict the actual model, now that I have no nan

In [44]:
x= np.asarray(withprice[predictors])
#x = sm.add_constant(x)
#.reshape((-1, 1))
model2 = sm.OLS(np.asarray(withprice[dependent]).reshape((-1, 1)), x)
fit = model2.fit()
#model.fit(x,np.asarray(withprice[dependent]).reshape((-1, 1)))
withprice['price_prediction'] = fit.predict(withprice[predictors])
results = model2.fit()

In [45]:
#x1= withprice[predictors].reshape((-1, 1))


In [52]:
model2 = sm.OLS(np.asarray(withprice[dependent]).reshape((-1, 1)), x)
fit = model2.fit()
#model.fit(x,np.asarray(withprice[dependent]).reshape((-1, 1)))


In [53]:
withprice['price_prediction'] = fit.predict(withprice[predictors])

In [55]:
results = model2.fit()

In [56]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.655
Model:                            OLS   Adj. R-squared (uncentered):              0.631
Method:                 Least Squares   F-statistic:                              27.26
Date:                Sat, 29 Apr 2023   Prob (F-statistic):                    4.92e-10
Time:                        14:07:32   Log-Likelihood:                         -414.77
No. Observations:                  46   AIC:                                      835.5
Df Residuals:                      43   BIC:                                      841.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [612]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [65]:
import sklearn as sklearn
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso

In [81]:
#predicted = withprice['price_prediction'].apply(lambda x: int(x))
#y_test = withprice['Land_Value'].apply(lambda x: int(x))

In [91]:
#precision, recall, fscore, support = score(y_test, predicted)
#precision = score(y_test, predicted)

#print('precision: {}'.format(precision))
#print('recall: {}'.format(recall))
#print('fscore: {}'.format(fscore))
#print('support: {}'.format(support))

#sklearn.metrics.precision_score(y_test, predicted)

In [112]:

from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso

In [113]:
model = Lasso(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# force scores to be positive
#scores = absolute(scores)
#print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

withprice2= withprice[['office_perc75', 'resid_share', 'Land_Value']]
data = withprice2.values
X, y = data[:,:-1 ], data[:, -1]
# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
model.fit(X,y)
withprice['Lasso_pred']= model.predict(X)

In [114]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# force scores to be positive
#scores = absolute(scores)
#print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

withprice2= withprice[['office_perc75', 'resid_share', 'Land_Value']]
data = withprice2.values
X, y = data[:,:-1 ], data[:, -1]
# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
model.fit(X,y)
withprice['Lasso_pred']= model.predict(X)

In [115]:
withprice2= withprice[['office_perc75', 'resid_share', 'Land_Value']]
data = withprice2.values

In [116]:
X, y = data[:,:-1 ], data[:, -1]


In [117]:
# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [120]:
model.fit(X,y)

Lasso()

In [128]:
withprice['Lasso_pred']= model.predict(X)

In [119]:
scores = abs(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean MAE: 1513.436 (687.199)
